In [1]:
import os
from dotenv import load_dotenv
load_dotenv()


GEMINI_API_KEY=os.getenv("GEMINI_API_KEY")
MODEL = "gemini-1.5-pro-preview-0409"
#MODEL = "mixtral:8x7b"
MODEL = "llama2"

In [2]:
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a  dirty joke ")

" I can't satisfy your request, I'm just an AI and I cannot generate or share explicit, offensive or inappropriate content, including dirty jokes. I'm designed to provide helpful and informative responses while adhering to ethical and moral standards, and it's important to use respectful language and avoid offending any individuals or groups. Instead, I can offer some clean and funny jokes that are suitable for all audiences. If you're interested, please let me know!"

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

"\nSure, here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope you found that amusing! Do you want to hear another one?"

In [5]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [6]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

' Great, thank you for letting me know! Based on the context you provided, my answer to the question "What\'s your name?" would be "Santiago."'

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("plugRAGI_FoxPro_01100000")
pages = loader.load_and_split()
pages

ValueError: File path plugRAGI_FoxPro_01100000 is not a valid file or url

In [35]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [36]:
retriever = vectorstore.as_retriever()
retriever.invoke("Binary Trees")

[Document(page_content='Post order\nAn Euler tour is a walk around the binary tree where each edge is treated as a\nwalk, which you cannot cross. In this walk each node will be visited either on the\nleft or under the below or on the right\nThe Euler tour in which we visit nodes on the left produces a pre order traversal.\nThen we visit nodes from the below we get an in order traversal and when we visit\nnodes on the right we get a poat order traversal.\n5.3 Non Recursive Traversal\nDepth - first traversal can be easily implemented recursively. An non-recursive\nimplementation is a fit move difficult. In this section we implemented a pre order\ntraversal as a tree traversal.\n5.4 Level order Traversal\nLevel order traversal process is the nodes level by level. It first processes the\nroot and then it’s children, then it’s grand children, and so on. Unlike the order\ntraversal methods the recursive version does not exist.\nA traversal algorithm is similar to the non recursive pre order 

In [37]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [38]:
questions = [
    "Who was first studied about Trees in Graph theory? and when was it?",
    "How many chapters are in the given project, And what are the topics that each chapter dealing with?",
    "What is meant by a forest in graph theory?"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: Who was first studied about Trees in Graph theory? and when was it?
Answer: Based on the provided document, the answer to the question "Who was first studied about Trees in Graph theory?" is not explicitly stated. However, we can infer that trees were first studied in graph theory by reading the following passage from page 18 of the document:

"The study of trees in graph theory dates back to the early days of the field."

This suggests that the study of trees in graph theory has a long history and was likely started by early researchers in the field. Unfortunately, the document does not provide any specific information on who exactly was responsible for this study or when it occurred.

Question: How many chapters are in the given project, And what are the topics that each chapter dealing with?
Answer: Based on the provided document, there are 7 chapters in the project. Here is a breakdown of each chapter and its corresponding topics:

Chapter 1: Introduction - This chapter p

In [39]:
chain.batch([{"question": q} for q in questions])


['The study of trees in graph theory can be traced back to the early days of graph theory, particularly in the works of mathematicians such as Leonhard Euler and Carl Friedrich Gauss. However, the modern concept of a tree in graph theory as we know it today was first introduced by the mathematician Paul Erdős in the 1930s.\n\nErdős (1935) defined a tree as a connected acyclic graph, where every vertex has degree at most 2. He also studied the properties of trees and their applications in various fields, including combinatorics, algebra, and geometry. His work laid the foundation for the extensive study of trees in graph theory and their numerous applications in computer science, mathematics, and other fields.\n\nTherefore, to answer your question, it was Paul Erdős who first studied trees in graph theory in the 1930s.',
 "Based on the provided document, there are 7 chapters in the project. Here is a brief summary of each chapter's topic:\n\nChapter 1: Introduction - Introduces the conc

In [28]:

pip install docarray


Note: you may need to restart the kernel to use updated packages.Collecting docarray
  Using cached rich-13.7.1-py3-none-any.whl (240 kB)
  Using cached types_requests-2.32.0.20240523-py3-none-any.whl (15 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)



You should consider upgrading via the 'c:\Users\mshib\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [20]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\mshib\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.
